In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1606684756221_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

In [24]:
datos = spark.read.csv('s3://aws-logs-351694975232-us-east-1/datasets/spark/Casos_positivos_de_COVID-19_en_Colombia.csv', inferSchema=True, header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
print(datos.count(), len(datos.columns))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6211 16

In [26]:
datos.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+--------------------+--------------------+---------------+-------------------+--------------------+-------------------+
|ID de caso|Fecha de notificación|Codigo DIVIPOLA|Ciudad de ubicación|Departamento o Distrito |  atención|Edad|Sexo|       Tipo|Estado| País de procedencia|                 FIS|Fecha de muerte|  Fecha diagnostico|    Fecha recuperado|  fecha reporte web|
+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+--------------------+--------------------+---------------+-------------------+--------------------+-------------------+
|         1|  2020-03-02 00:00:00|          11001|        Bogotá D.C.|             Bogotá D.C.|Recuperado|  19|   F|  Importado|  Leve|              Italia|2020-02-27T00:00:...|          -   -|2020-03-06 00:00:00|2020-03-13T00:00:...|2

In [27]:
#Definir la funcion para agregar las columnas

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
def edad_etapa(edad):
    etapa = ''
    if (edad<=5):
        etapa = 'primera infancia'
    elif (5<edad<=11):
        etapa = 'infancia'
    elif (12<edad<=18):
        etapa = 'adolescencia'
    elif (18<edad<=26):
        etapa = 'juventud'
    elif (26<edad<=59):
        etapa = 'adultez'
    elif (edad>=60):
        etapa = 'vejez'
    return etapa

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
datos.select('edad').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+
|edad|
+----+
|  19|
|  34|
|  50|
|  55|
|  25|
|  27|
|  85|
|  22|
|  28|
|  36|
|  42|
|  74|
|  68|
|  48|
|  30|
|  61|
|  73|
|  54|
|  54|
|  26|
+----+
only showing top 20 rows

In [35]:
#Agregar Columna
etapa_udf = spark.udf.register('Etapas', edad_etapa)

datos.withColumn('Etapa', etapa_udf(datos['Edad'])).show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+--------------------+--------------------+---------------+-------------------+--------------------+-------------------+--------+
|ID de caso|Fecha de notificación|Codigo DIVIPOLA|Ciudad de ubicación|Departamento o Distrito |  atención|Edad|Sexo|       Tipo|Estado| País de procedencia|                 FIS|Fecha de muerte|  Fecha diagnostico|    Fecha recuperado|  fecha reporte web|   Etapa|
+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+--------------------+--------------------+---------------+-------------------+--------------------+-------------------+--------+
|         1|  2020-03-02 00:00:00|          11001|        Bogotá D.C.|             Bogotá D.C.|Recuperado|  19|   F|  Importado|  Leve|              Italia|2020-02-27T00:00:...|          -   -|2020-03-06 00:0

In [36]:
#Eliminar Columna
datos2 = datos.drop('fecha reporte web')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
datos2.show(6)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+-------------------+--------------------+---------------+-------------------+--------------------+
|ID de caso|Fecha de notificación|Codigo DIVIPOLA|Ciudad de ubicación|Departamento o Distrito |  atención|Edad|Sexo|       Tipo|Estado|País de procedencia|                 FIS|Fecha de muerte|  Fecha diagnostico|    Fecha recuperado|
+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+-------------------+--------------------+---------------+-------------------+--------------------+
|         1|  2020-03-02 00:00:00|          11001|        Bogotá D.C.|             Bogotá D.C.|Recuperado|  19|   F|  Importado|  Leve|             Italia|2020-02-27T00:00:...|          -   -|2020-03-06 00:00:00|2020-03-13T00:00:...|
|         2|  2020-03-06 00:00:00|          76111|Guadalajara de

In [40]:
#Filtrado de datos
datos.select('Edad', 'Estado', 'atención').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+---------+------------+
|Edad|   Estado|    atención|
+----+---------+------------+
|  82|    Grave|Hospital UCI|
|  49|     Leve|        Casa|
|  54|     Leve|  Recuperado|
|  66|     Leve|  Recuperado|
|  82|     Leve|  Recuperado|
|  34|    Grave|Hospital UCI|
|  47| Moderado|  Recuperado|
|  80|     Leve|  Recuperado|
|  27| Moderado|    Hospital|
|  46|     Leve|  Recuperado|
|  74|     Leve|  Recuperado|
|  53|     Leve|  Recuperado|
|  91|Fallecido|   Fallecido|
|  35|     Leve|        Casa|
|  79|    Grave|Hospital UCI|
|  76|     Leve|        Casa|
|  29| Moderado|    Hospital|
|  58|    Grave|Hospital UCI|
|  84|Fallecido|   Fallecido|
|  16|     Leve|  Recuperado|
+----+---------+------------+
only showing top 20 rows

In [43]:
datos.select('Fecha diagnostico', 'Fecha recuperado', 'Estado').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+--------+
|  Fecha diagnostico|    Fecha recuperado|  Estado|
+-------------------+--------------------+--------+
|2020-03-24 00:00:00|2020-04-13T23:59:...|    Leve|
|2020-03-27 00:00:00|               -   -|Moderado|
|2020-04-07 00:00:00|2020-04-14T23:59:...|    Leve|
|2020-04-05 00:00:00|2020-04-14T00:00:...|Moderado|
|2020-04-01 00:00:00|2020-04-08T00:00:...|    Leve|
|2020-04-08 00:00:00|2020-04-04T00:00:...|    Leve|
|2020-04-03 00:00:00|2020-04-12T00:00:...|Moderado|
|2020-03-25 00:00:00|2020-04-06T00:00:...|    Leve|
|2020-03-20 00:00:00|               -   -|    Leve|
|2020-04-12 00:00:00|2020-04-21T00:00:...|    Leve|
|2020-04-02 00:00:00|2020-04-23T00:00:...|    Leve|
|2020-04-09 00:00:00|2020-04-16T23:59:...|    Leve|
|2020-04-12 00:00:00|               -   -|Moderado|
|2020-04-12 00:00:00|2020-04-24T23:59:...|    Leve|
|2020-03-30 00:00:00|2020-04-18T00:00:...|    Leve|
|2020-04-15 00:00:00|2020-04-25T23:59:...|    Leve|
|2020-03-25 

In [45]:
datos.groupBy('Sexo').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+
|Sexo|count|
+----+-----+
|   M| 3290|
|   f|    3|
|   F| 2917|
|   m|    1|
+----+-----+

In [47]:
datos.groupBy('País de procedencia').count().orderBy('count', ascending=False).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
| País de procedencia|count|
+--------------------+-----+
|            Colombia| 5401|
|              España|  258|
|Estados Unidos de...|  206|
|             Ecuador|   47|
|              México|   46|
|              Panamá|   34|
|              Brasil|   30|
|             Turquía|   28|
|             Jamaica|   16|
|             Francia|   15|
|         Reino Unido|   14|
|         Puerto Rico|   13|
|              Egipto|   11|
|            Alemania|   10|
|              Italia|    9|
|República Dominicana|    9|
|                Perú|    9|
|San Martín (Francia)|    7|
|              Israel|    5|
|              Grecia|    4|
+--------------------+-----+
only showing top 20 rows

In [48]:
url = 's3://aws-logs-351694975232-us-east-1/notebooks/laboratorio/resultados/covid'

datos.coalesce(1).write.format('csv').option('header','True').save(url)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…